In [81]:
import random
def is_valid(board, row, col, num):
    # check  row and column
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    # check  3x3 grid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[i + start_row][j + start_col] == num:
                return False

    return True

def find_empty_location(board):
    min_values = float('inf')
    min_row, min_col = -1, -1

    # find the cell with the minimum remaining values
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                count = sum(1 for num in range(1, 10) if is_valid(board, i, j, num))
                if count < min_values:
                    min_values = count
                    min_row, min_col = i, j

    return min_row, min_col

def arc_consistency(board, arcs):
    while arcs:
        # taking one arc from list of arcs
        row, col = arcs.pop(0)

        # for rows
        for i in range(9):
            if i != col and board[row][i] == 0:
                domain = set(range(1, 10))
                
                for j in range(9):
                    if j != i and board[row][j] != 0:
                        domain.discard(board[row][j])
                
                # if there are no valid values  return False
                if not domain:
                    return False
                
                # if only one value left in the domain we assign it 
                if len(domain) == 1:
                    board[row][i] = domain.pop()
                    # Add arcs for the affected cells in the same row
                    arcs.extend([(row, x) for x in range(9) if x != i])
        
        # for columns
        for i in range(9):
            if i != row and board[i][col] == 0:
                domain = set(range(1, 10))
                
                for j in range(9):
                    if j != i and board[j][col] != 0:
                        domain.discard(board[j][col])
                
                if not domain:
                    return False
                
                # if there is only one value left in the domain we assign it
                if len(domain) == 1:
                    board[i][col] = domain.pop()
                    # Add arcs for the affected cells in the same column
                    arcs.extend([(x, col) for x in range(9) if x != i])
    
    return True


def solve_sudoku(board):
    arcs = [(i, j) for i in range(9) for j in range(9) if board[i][j] == 0]
    if not arc_consistency(board, arcs):
        return False

    row, col = find_empty_location(board)
    if row == -1 and col == -1:
        return True  # If no empty location sudoku solved

    nums = [(num, sum(1 for i in range(1, 10) if is_valid(board, row, col, i))) for num in range(1, 10)]
    nums.sort(key=lambda x: x[1])

    for num, _ in nums:
        if is_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku(board):
                return True
            board[row][col] = 0  # backtracking

    return False


def is_valid_board(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                continue
            num = board[i][j]
            board[i][j] = 0
            if not is_valid(board, i, j, num):
                return False
            board[i][j] = num
    return True

def generate_random_board():
    while True:
        board = [[0 for i in range(9)] for j in range(9)]
        numbers_filled =random.randint(17, 25)
        for i in range(numbers_filled):  
            while True:
                row, col = random.randint(0, 8), random.randint(0, 8)
                if board[row][col] == 0:
                    num = random.randint(1, 9)
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        break
        # Check if the generated board is valid
        if is_valid_board(board):
            return board


def print_sudoku(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(board[i][j], end=" ")
        print()


board = generate_random_board()        
print("Sudoku board:")
print_sudoku(board)

print ("\nSolved Sudoku board:")
if solve_sudoku(board):
    print_sudoku(board)
else:
    print("No solution exists!")


Sudoku board:
0 0 9 | 0 8 0 | 3 0 4 
0 5 0 | 0 0 0 | 0 8 0 
3 0 0 | 0 9 1 | 0 0 0 
---------------------
0 0 8 | 0 0 0 | 0 7 2 
6 7 0 | 0 0 0 | 0 0 1 
9 0 5 | 0 0 0 | 0 0 0 
---------------------
0 0 6 | 0 1 2 | 0 0 0 
0 0 0 | 9 0 0 | 5 0 3 
0 0 0 | 0 5 0 | 0 4 0 

Solved Sudoku board:
2 6 9 | 5 8 7 | 3 1 4 
7 5 1 | 3 4 6 | 2 8 9 
3 8 4 | 2 9 1 | 7 6 5 
---------------------
1 4 8 | 6 3 5 | 9 7 2 
6 7 3 | 8 2 9 | 4 5 1 
9 2 5 | 1 7 4 | 6 3 8 
---------------------
5 3 6 | 4 1 2 | 8 9 7 
4 1 7 | 9 6 8 | 5 2 3 
8 9 2 | 7 5 3 | 1 4 6 


In [109]:
import random

def calculate_fitness(data):
    fitness = 0
    # Check rows and columns
    for i in range(3):
        fitness += abs(15 - sum(data[i][j] for j in range(3)))  
        fitness += abs(15 - sum(data[j][i] for j in range(3)))  
    # Check diagonals
    fitness += abs(15 - sum(data[i][i] for i in range(3)))  
    fitness += abs(15 - sum(data[i][2-i] for i in range(3)))  
    return fitness


def crossover(parent1, parent2):
    child_data = [[0]*3 for i in range(3)]
    for i in range(3):
        for j in range(3):
            if random.random() < 0.5:
                child_data[i][j] = parent1[i][j]
            else:
                child_data[i][j] = parent2[i][j]

    # convert to 1d array to check missing and duplicate values
    child_1d = [child_data[i][j] for i in range(3) for j in range(3)]

    # check missing and duplicate values in the child 
    missing = set(range(1, 10)) - set(child_1d)
    duplicates = set([x for x in child_1d if child_1d.count(x) > 1])
    
    # Replace duplicates with missing values
    for dup in duplicates:
        child_1d[child_1d.index(dup)] = missing.pop()
    # convert to 2d again
    for i in range(3):
        for j in range(3):
            child_data[i][j] = child_1d[i*3 + j]
    return child_data

def mutate(individual):
    # Select two distinct random positions in the solution grid
    i, j, k, l = random.sample(range(3), 2) + random.sample(range(3), 2)
    # Swap the values at the selected positions
    individual[i][j], individual[k][l] = individual[k][l], individual[i][j]
    return individual

def genetic_algorithm():
    population = [[random.sample(range(1, 10), 9) for i in range(3)] for j in range(1000)]
    generations = 1000

    for generation in range(generations):
        # Sort population based on fitness score lowest to highest
        population.sort(key=lambda x: calculate_fitness(x))

        if population and calculate_fitness(population[0]) == 0:
            return population[0], generation

        new_population = []

        # keep 10 % of population same
        s = int(0.1 * len(population))
        new_population.extend(population[:s])

        # do crossover for 50% population
        s = int(0.5 * len(population))
        for i in range(s):
            parent1 = random.choice(population[:50])
            parent2 = random.choice(population[:50])
            child = crossover(parent1, parent2)
            new_population.append(child)

        # do mutation for the rest 
        for individual in new_population[s:]:
            if random.random() < 0.1:
                mutate(individual)

        population = new_population

    return None, generations

# Function to print the Sudoku solution
def print_solution(solution):
    
    if solution:
        
        for row in solution:
            print(row)
    else:
        print("No solution found")

# Run the genetic algorithm and print the solution
solution, generations = genetic_algorithm()
print("Solution found in generation:", generations)
print_solution(solution)


Solution found in generation: 5
[4, 9, 2]
[3, 5, 7]
[8, 1, 6]
